In [90]:
pip install contractions 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Jalon 3 : Développement application web locale


In [91]:
import pickle
import numpy as np
import pandas as pd
import numpy as np
import os
import seaborn as sns
import en_core_web_sm
import nltk
import contractions

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from pickle import *
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS

In [92]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [93]:
tokenizer = RegexpTokenizer(r'\w+')

def tokenize_text(text):
    text_processed = " ".join(tokenizer.tokenize(text))
    return text_processed

In [94]:
nlp = en_core_web_sm.load(disable=['parser', 'tagger', 'ner'])

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    lemmatized_text_list = list()
    
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatized_text_list.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    
    return " ".join(lemmatized_text_list)


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'tagger', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [95]:
def normalize_text(text):
    return " ".join([word.lower() for word in text.split()])

In [96]:
def contraction_text(text):
    return contractions.fix(text)

In [97]:
negative_words = ['not', 'no', 'never', 'nor', 'hardly', 'barely']
negative_prefix = "NOT_"

def get_negative_token(text):
    tokens = text.split()
    negative_idx = [i+1 for i in range(len(tokens)-1) if tokens[i] in negative_words]
    for idx in negative_idx:
        if idx < len(tokens):
            tokens[idx]= negative_prefix + tokens[idx]
    
    tokens = [token for i,token in enumerate(tokens) if i+1 not in negative_idx]
    
    return " ".join(tokens)

In [98]:
def remove_stopwords(text):
    english_stopwords = stopwords.words("english") + list(STOP_WORDS) + ["tell", "restaurant"]
    
    return " ".join([word for word in text.split() if word not in english_stopwords])

In [99]:
def preprocess_text(text):
    
    # Tokenize review
    text = tokenize_text(text)
    
    # Lemmatize review
    text = lemmatize_text(text)
    
    # Normalize review
    text = normalize_text(text)
    
    # Remove contractions
    text = contraction_text(text)

    # Get negative tokens
    text = get_negative_token(text)
    
    # Remove stopwords
    text = remove_stopwords(text)
    
    return text  

#Partie 1 : Prédiction d'un sujet d'insatisfaction

In [100]:
#Importer les fichiers pickles 'model' et 'vectoriseur' 

model_pickle = open ("/content/drive/MyDrive/E3/TLN/modelEntraineLehna",'rb')
modelEntraine = load(model_pickle)
print(modelEntraine)

vectoriseur_pickle = open ("/content/drive/MyDrive/E3/TLN/vectoriseurLehna",'rb')
vectorizer = load(vectoriseur_pickle)
print(vectorizer)

NMF(n_components=15)
TfidfVectorizer(max_df=0.8, min_df=0.01)


In [101]:
def prediction(model,vectorizer,n_topic,new_reviews):
  new_reviews = preprocess_text(new_reviews)
  blob=TextBlob(new_reviews)
  sentimentBlob=blob.sentiment.polarity
  new_reviews = [new_reviews]
  new_reviews_transformed=vectorizer.transform(new_reviews)

  prediction= model.transform(new_reviews_transformed)
 
  topics=[ 'Cadre du lieu',
           'Plats en sauce',
           'Menu pizza ',
           'Service livraison et commandes',
           'Qualité des plats ',
           'Qualité du service',
           'Menu burger',
           'Temps attente',
           'Menu chicken',
           'Service bar ',
           'Localisation du lieu',
           'Relation client',
           'Menu sandwich',
           'Menu sushis',
           'Clients revenus']

  if sentimentBlob<0 and sentimentBlob>-1:
    max = np.argsort(prediction)
    max_list=(list(max[0]))
    max_list.reverse()
    print(max_list)
    topic=[]
    for i in range(n_topic):
      topic.append(topics[max_list[i]])  
    return sentimentBlob,prediction,topic

  return sentimentBlob

In [102]:
new_reviews = "I dont like chicken "
prediction(modelEntraine,vectorizer,2,new_reviews)

[8, 14, 13, 12, 11, 10, 9, 7, 6, 5, 4, 3, 2, 1, 0]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"


(-0.6,
 array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.1314366, 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       ]]),
 ['Menu chicken', 'Clients revenus'])

In [103]:
TOPICS = ({     0 :'Cadre du lieu',
                1 :'Plats en sauce',
                2 :'Menu pizza ',
                3 :'Service livraison et commandes',
                4 :'Qualité des plats ',
                5 :'Qualité du service',
                6 :'Menu burger',
                7 :'Temps attente',
                8 :'Menu chicken',
                9 :'Service bar ',
                10:'Localisation du lieu',
                11:'Relation client',
                12:'Menu sandwich',
                13:'Menu sushis',
                14:'Clients revenus'
          })

def predict_topics(model, vectorizer, n_topics, text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity < 0:
        text = preprocess_text(text)
        text = [text]
        vectorized = vectorizer.transform(text)
        topics_correlations = model.transform(vectorized)
        unsorted_topics_correlations = topics_correlations[0].copy()
        topics_correlations[0].sort()
        sorted = topics_correlations[0][::-1]
        print(sorted)
        topics = []
        for i in range(n_topics):
            corr_value = sorted[i]
            result = np.where(unsorted_topics_correlations == corr_value)[0]
            topics.append(TOPICS.get(result[0]))
        print(topics)
        return polarity
    else:
        return polarity

In [104]:
new_reviews = "I dont like chicken "
predict_topics(modelEntraine,vectorizer,2,new_reviews)

[0.1314366 0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.       ]
['Menu chicken', 'Cadre du lieu']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"


-0.6